# Load an edge table of gene similarities into NDEx

This tutorial shows how to convert an edge table containing similarity weights for genes with the following format:

```Bash
SOURCE TARGET WEIGHT
GENE1  GENE2  0.123
GENE1  GENE3  0.144
.
.
```

to an adjacency matrix as well as how to upload this data to NDEx. 

**WARNING:** Large tables take lots of ram. For example running this workflow with an edge table with 180 million rows containing around 19,000 genes will consume **????** gigabytes of ram and will take ?? - ?? minutes to run

This tutorial requires the following packages

* python-igraph (Best to install via Conda ie: `conda install -c conda-forge python-igraph`)
* ddot
* ndex2
* simplejson
* pandas
* numpy


# Import needed modules

In [ ]:
import os
import sys
import getpass

import ddot
import ndex2
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix 


# Load edge table via Pandas

Enter path to edge table file. It is assumed this file has the following header line: `SOURCE TARGET WEIGHT`

Example: `/tmp/foo.tsv`

In [ ]:
# sys.version_info gets the version of python, needed to use correct call to get user input
sys.stdout.write('Enter path to edge table file:\n')
if sys.version_info[0] >= 3:
   edgetable = os.path.abspath(input())
else:
   edgetable = os.path.abspath(raw_input())


In [ ]:
sys.stdout.write('Edge table file set to: ' + edgetable + '\n')

In [ ]:
# Loads table into Pandas, for a table of 180 million rows this takes a minute or two and
# requires ~6gb of ram
df = pd.read_csv(edgetable, sep='\t', dtype={'WEIGHT': np.float64})

df.head()

# Get sorted gene list and mapping

The `coo_matrix` function cannot work with numbers for labels so the code below creates a mapping of gene names to integers, storing the sorted list of genes in `genes` and the mapping in `gene_map_list` which is a list of tuples `(#, GENE NAME)`

In [ ]:
genes = sorted(list(set(df['SOURCE'].append(df['TARGET']))))

gene_map_list = [(i, genes[i]) for i in range(len(genes))]

gene_map_list    

genes

# Replace gene names in dataframe with integers

In [ ]:
for i in range(len(gene_map_list)):
    df.replace(gene_map_list[i][1], gene_map_list[i][0], inplace=True)

df

# Create adjacency matrix 

In [ ]:
sparse_mat = coo_matrix((df.iloc[:,2], (df.iloc[:,0],df.iloc[:,1])), shape=(len(nodes), len(nodes)))

full_mat = sparse_mat + sparse_mat.T

full_mat

# Create NDEx NiceCXNetwork object

NDEx utilizes CX format for storage of data. The next command converts the matrix data into this format

In [ ]:
network = ddot.utils.create_edgeMatrix(full_mat.todense(), genes, genes,verbose=True,ndex2=True)

# sets the name of the network
network.set_name('test similarity network')
network

# Get NDEX credentials

In [ ]:
sys.stdout.write('Enter NDEx username:\n')
if sys.version_info[0] >= 3:
    user = input()
else:
    user = raw_input()

sys.stdout.write('Enter NDEx password:\n')
password = getpass.getpass()


In [ ]:
# NDEx server to use for production use public.ndexbio.org
server_url = 'test.ndexbio.org'

res = network.upload_to(server_url, user, password)

sys.stdout.write('If successful the value below will be low level URL\n')
sys.stdout.write('The network will be private by default so\n')
sys.stdout.write('to see the network visit http://' + server_url + ' and login with user account entered earlier\n')

res

In [ ]:
print('Tutorial complete. Have a nice day.')